<a href="https://colab.research.google.com/github/akhiilkasare/Sentiment-Analysis-Aarogya-Setu-App-using-BERT/blob/main/Web_Scrapping_reviews_for_Aarogya_Setu_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq google-play-scraper

     |████████████████████████████████| 51kB 3.0MB/s 


In [2]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [3]:
app_packages = [
      'nic.goi.aarogyasetu'
]

In [4]:
app_info = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='in')
  del info['comments']
  app_info.append(info)

100%|██████████| 1/1 [00:00<00:00,  5.66it/s]


In [5]:
def print_json(json_object):
  json_str = json.dumps(
      json_object,
      indent=2,
      sort_keys=True,
      default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [6]:
print_json(app_info[0])

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 and up",
  "appId": "nic.goi.aarogyasetu",
  "containsAds": false,
  "contentRating": "Rated for 3+",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "Aarogya Setu is a mobile application developed by the Government of India to connect essential health services with the people of India in our combined fight against COVID-19. The App is aimed at augmenting the initiatives of the Government of India, particularly the Department of Health, in proactively reaching out to and informing the users of the app regarding risks, best practices and relevant advisories pertaining to the containment of COVID-19.",
  "descriptionHTML": "Aarogya Setu is a mobile application developed by the Government of India to connect essential health services with the people of India in our combined fight against COVID-19. The App is aimed at augmenting the initiatives of the Government of India, particularly

In [7]:
app_info_df = pd.DataFrame(app_info)
app_info_df.head()

,title,description,descriptionHTML,summary,summaryHTML,installs,minInstalls,score,ratings,reviews,histogram,price,free,currency,sale,saleTime,originalPrice,saleText,offersIAP,inAppProductPrice,size,androidVersion,androidVersionText,developer,developerId,developerEmail,developerWebsite,developerAddress,privacyPolicy,developerInternalID,genre,genreId,icon,headerImage,screenshots,video,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,appId,url
0,Aarogya Setu,Aarogya Setu is a mobile application developed...,Aarogya Setu is a mobile application developed...,GoI’s app to connect health services with the ...,GoI’s app to connect health services with the ...,"100,000,000+",100000000,4.163772,1401318,433244,"[181844, 44959, 81322, 146922, 946271]",0,True,USD,False,None,None,None,False,None,4.2M,5.0,5.0 and up,NIC eGov Mobile Apps,9076108670215860604,support.aarogyasetu@gov.in,None,NIC,https://web.swaraksha.gov.in/ncv19/privacy/,9076108670215860604,Health & Fitness,HEALTH_AND_FITNESS,https://play-lh.googleusercontent.com/2FV06dPO...,https://play-lh.googleusercontent.com/qonFxIPe...,[https://play-lh.googleusercontent.com/EZUnZbm...,None,None,Rated for 3+,None,None,False,"Apr 1, 2020",1594221933,1.4.1,None,None,nic.goi.aarogyasetu,https://play.google.com/store/apps/details?id=...


In [8]:
app_info_df.to_csv('apps.csv', index=None, header=True)

In [9]:
df = pd.read_csv('/content/apps.csv')
df.head()

,title,description,descriptionHTML,summary,summaryHTML,installs,minInstalls,score,ratings,reviews,histogram,price,free,currency,sale,saleTime,originalPrice,saleText,offersIAP,inAppProductPrice,size,androidVersion,androidVersionText,developer,developerId,developerEmail,developerWebsite,developerAddress,privacyPolicy,developerInternalID,genre,genreId,icon,headerImage,screenshots,video,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,appId,url
0,Aarogya Setu,Aarogya Setu is a mobile application developed...,Aarogya Setu is a mobile application developed...,GoI’s app to connect health services with the ...,GoI’s app to connect health services with the ...,"100,000,000+",100000000,4.163772,1401318,433244,"[181844, 44959, 81322, 146922, 946271]",0,True,USD,False,NaN,NaN,NaN,False,NaN,4.2M,5.0,5.0 and up,NIC eGov Mobile Apps,9076108670215860604,support.aarogyasetu@gov.in,NaN,NIC,https://web.swaraksha.gov.in/ncv19/privacy/,9076108670215860604,Health & Fitness,HEALTH_AND_FITNESS,https://play-lh.googleusercontent.com/2FV06dPO...,https://play-lh.googleusercontent.com/qonFxIPe...,['https://play-lh.googleusercontent.com/EZUnZb...,NaN,NaN,Rated for 3+,NaN,NaN,False,"Apr 1, 2020",1594221933,1.4.1,NaN,NaN,nic.goi.aarogyasetu,https://play.google.com/store/apps/details?id=...


In [13]:
app_reviews = []
for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='in',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)


100%|██████████| 1/1 [00:04<00:00,  4.94s/it]


In [14]:
print_json(app_reviews[0])

{
  "appId": "nic.goi.aarogyasetu",
  "at": "2020-10-30 20:22:17",
  "content": "Totally useless app. If it has real time data on positive patients and their location, with my Bluetooth always on I expect that it warns me when I am in the vicinity. It does not. Moreover, now that I have 2 next door, it has no clue, it does not show them. It caused undue stress on me telling me I am at risk, two days after I may have crossed someone in a busy intersection. It is simply, ineffective, useless. I am not removing it only because some places want to check it.",
  "repliedAt": "2020-10-31 04:27:48",
  "replyContent": "The  Aarogya Setu App detects other devices with app installed that have come in the Bluetooth / GPS proximity of your phone and captures this information. In the unfortunate case of any of those recent contacts Test positive, the App calculates your risk of infection based on recency and proximity of your interaction and recommends suitable action",
  "reviewCreatedVersion": "1

In [16]:
apps_reviews_df = pd.DataFrame(app_reviews)
apps_reviews_df.shape

(1200, 12)

In [17]:
apps_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOFdWMJ_oyeoIT_wOSafcZLDc9EquKXZI9YOwOq...,MOHAMMED Ali,https://play-lh.googleusercontent.com/-gAOtym8...,Totally useless app. If it has real time data ...,1,111,1.4.1,2020-10-30 20:22:17,The Aarogya Setu App detects other devices wi...,2020-10-31 04:27:48,most_relevant,nic.goi.aarogyasetu
1,gp:AOqpTOEoCVAknSqa9iSs5AbhL9t0f_8cKmxlFiIRG1H...,Ravi Kumar Reddy Bandi,https://play-lh.googleusercontent.com/a-/AOh14...,"Not at all usefull, it isn't a dynamic app whi...",1,87,1.4.1,2020-10-30 08:28:39,None,NaT,most_relevant,nic.goi.aarogyasetu
2,gp:AOqpTOGAU_JskSqjNwAajeOQpGIwDj0z1lFAkZlkMpD...,Ranjan Bharadvaj,https://play-lh.googleusercontent.com/a-/AOh14...,"Initially it was worth having it, could see th...",1,9,1.4.1,2020-10-31 02:25:55,None,NaT,most_relevant,nic.goi.aarogyasetu
3,gp:AOqpTOGlLEzm67t1PpjCzwcWLKDwJgS-1fTpJfsnW67...,Pravin's Cuisine,https://play-lh.googleusercontent.com/a-/AOh14...,Totally useless app. If it has real time data ...,1,4245,1.4.1,2020-09-12 13:18:18,Thanks for your feedback! We deeply respect yo...,2020-09-12 13:44:14,most_relevant,nic.goi.aarogyasetu
4,gp:AOqpTOHyMTgfF6-a5S8tIY-48AfKKs-2OuhydFlN327...,Johnson P,https://play-lh.googleusercontent.com/-Qd2pEwz...,There is no change in opinion. In fact it is g...,1,313,1.4.1,2020-10-20 17:45:19,We are sorry that we didn't meet your expectat...,2020-07-04 04:51:33,most_relevant,nic.goi.aarogyasetu


In [18]:
apps_reviews_df.to_csv('reviews.csv', index=None, header=True)